In [ ]:
# Load Packages - KEEP MINIMAL FOR FISHTANK COMPATIBILITY
import pandas as pd
import numpy as np
import os
import sys
import time

In [ ]:
# Doesn't work anymore??
# from particle import Particle

# HOTFIX
class Particle:
    def __init__(self):
        pass
    def from_pdgid(pdgid):
        P = Particle()
        masses_table = {
            2212:938.27208816,
            1000020040:3728.4013213,
            11:0.51099895,
            22:0
        }
        P.pdgid = pdgid
        P.mass = masses_table[pdgid]
        return P

In [82]:
# set locations for working files
if len(sys.argv) != 3:
    print("Usage: python3 iter-params.py <automation_dir> <attpcroot_dir>")
    print('Assuming testing directories')
    automation_dir = '/mnt/analysis/e17023/Adam/.sims/0/'
    attpcroot_dir = automation_dir + 'ATTPCROOTv2/'
else:
    # Automation directory
    automation_dir = sys.argv[1]
    
    # ATTPCROOTv2 directory
    attpcroot_dir = sys.argv[2]

Usage: python3 iter-params.py <automation_dir> <attpcroot_dir>
Assuming testing directories


In [83]:
def indicator_file(file_type, indicator_directory=automation_dir):
    # remove old indicator file(s)
    for file in os.listdir(indicator_directory):
        if file.endswith('.tmp'):
            os.remove(indicator_directory + file)
    
    with open(indicator_directory + file_type + '.tmp', 'w') as f:
        f.write('1')
    if file_type == 'STOP':
        print('STOPPING')
        sys.exit()
    return None

In [84]:
def stop_check(indicator_directory):
    if os.path.isfile(indicator_directory + 'STOP.tmp'):
        print('Stop file found. Stopping.')
        indicator_file('STOP')
        sys.exit(0)
    if os.path.isfile(indicator_directory + '../STOP.tmp'):
        print('Master stop file found. Stopping.')
        indicator_file('STOP')
        sys.exit(0)
    return None

In [85]:
def pdgid_lookup(P):
    lookup = {
        # for short-hand notations of particles
        'p': 2212, # proton
        'proton' : 2212,
        'a': 1000020040, # alpha
        'alpha' : 1000020040,
        'b' : 11, # beta
        'beta' : 11,
        'e' : 11, # electron
        'electron' : 11,
        'g' : 22, # gamma
        'gamma' : 22,
    }
    try:
        int(P) # check if P is an integer (PDGID)
        return P
    except ValueError:
        return lookup[P.lower()]
    except KeyError:
        indicator_file('STOP')
        raise Exception('Error: Unsupported particle type specified')

In [86]:
def energy_to_momentum(energy, particle):
    # input energy in KeV, convert to MeV
    energy = energy/1000
    
    pdgid = pdgid_lookup(particle)
    P = Particle.from_pdgid(pdgid)
    mass = P.mass # MeV/c^2 (rest mass)
    
    momentum = np.sqrt(2*mass*energy)/1000 # GeV/c
    return momentum

In [87]:
def write_gadgetsimpar(parameters, active_sim, automation_dir=automation_dir, attpcroot_dir=attpcroot_dir):
    with open(automation_dir + 'GADGET.sim.par', 'r') as file:
        filedata = file.readlines()
    for param in parameters.columns:
        for i, line in enumerate(filedata):
            if param == line.split(':')[0]:
                # Line composition:     param:ptype_t   paramval   # units / comments
                ptype = line.split(':')[1].split('_')[0]
                paramval = parameters.loc[active_sim, param]
                filedata[i] = param + ': ' + ptype + '_t     ' + str(paramval) + '     #' + line.split('#')[1]
    with open(attpcroot_dir + 'parameters/GADGET.sim.par', 'w') as file:
        file.writelines(filedata)
    return None

In [88]:
def write_AtTPC20MgDecay_pag(parameters, active_sim, automation_dir=automation_dir, attpcroot_dir=attpcroot_dir):
    P0 = parameters.loc[active_sim, 'P0']; E0 = parameters.loc[active_sim, 'E0']
    P1 = parameters.loc[active_sim, 'P1']; E1 = parameters.loc[active_sim, 'E1']
    b2b = False
    if str(P1).startswith('-'):
        b2b = True # back-to-back event
        P1 = str(P1)[1:]
    with open(automation_dir + 'AtTPC20MgDecay_pag.cxx', 'r') as file :
        filedata = file.readlines()
    for i, line in enumerate(filedata):
        if 'P0' in line.split('//')[-1]:
            filedata[i] = line.split('=')[0] + '= ' + str(pdgid_lookup(P0)) + '; // P0\n'
        if 'E0' in line.split('//')[-1]:
            filedata[i] = line.split('=')[0] + '= ' + str(energy_to_momentum(E0, P0)) + '; // E0\n'

        if 'P1' in line.split('//')[-1]:
            filedata[i] = line.split('=')[0] + '= ' + str(pdgid_lookup(P1)) + '; // P1\n'
        if 'E1' in line.split('//')[-1]:
            filedata[i] = line.split('=')[0] + '= ' + str(energy_to_momentum(E1, P1)) + '; // E1\n'

        if "Seed" in line.split('//')[-1]:
            if 'Seed' in parameters.columns:
                if parameters.loc[active_sim, 'Seed'] != 0:
                    filedata[i] = line.split('(')[0] + '(' + str(parameters.loc[active_sim, 'Seed']) + '); // Seed\n'

        if 'Efficiency Fix' in line.split('//')[-1]: # place event origin on opposite side of detector to avoid efficiency issues
            if 'Xb' in parameters.columns:
                if parameters.loc[active_sim, 'Xb'] != 99:
                    filedata[i] = '// ' + line # comment out line if Xb is not 99 (default behavior unless specified)
            else:
                filedata[i] = '// ' + line # comment out line if Xb isn't specified (default behavior)

        if 'b2b' in line.split('//')[-1]:
            if b2b: # back-to-back event
                if 'thetaAlpha' in line: # reference and flip proton theta
                    filedata[i] = line.split('=')[0] + '= -1 * ' + line.split('=')[1].split('(')[0] + '(thetaProton); // b2b\n'
                if 'phiAlpha' in line: # reference proton phi
                    filedata[i] = line.split('(')[0] + '(phiProton); // b2b\n'
    
    with open(attpcroot_dir + "AtGenerators/AtTPC20MgDecay_pag.cxx" , 'w') as file:
        file.writelines(filedata)
    return None

In [89]:
def write_Mg20_test_sim(parameters, active_sim, automation_dir=automation_dir, attpcroot_dir=attpcroot_dir):
    with open(automation_dir + 'Mg20_test_sim_pag.C', 'r') as file:
        filedata = file.readlines()
    E0 = parameters.loc[active_sim, 'E0']; E1 = parameters.loc[active_sim, 'E1']
    
    # Modify particle momentum
    for i, line in enumerate(filedata):
        if 'P0 E0' in line.split('//')[-1]:
            filedata[i] = line.split('(')[0] + '(' + str(E0 / 1000000) + ', 1); // P0 E0\n'

        if 'P1 E1' in line.split('//')[-1]:
            if E1 == 0:
                filedata[i] = '// ' + line
            else:
                filedata[i] = line.split('(')[0] + '(' + str(E1 / 1000000) + ', 1); // P1 E1\n'

    # modify particle origin
    for i, line in enumerate(filedata):
        if 'bounds' in line.split('//')[-1]:
            Xb1 = -2; Xb2 = 2; Yb1 = -2; Yb2 = 2; Zb1 = 10; Zb2 = 40
            if 'Xb' in parameters.columns:
                Xb1 = -1*parameters.loc[active_sim, 'Xb']
                Xb2 = parameters.loc[active_sim, 'Xb']
            if 'Yb' in parameters.columns:
                Yb1 = -1*parameters.loc[active_sim, 'Yb']
                Yb2 = parameters.loc[active_sim, 'Yb']
            if 'Zb1' in parameters.columns:
                Zb1 = parameters.loc[active_sim, 'Zb1']
            if 'Zb2' in parameters.columns:
                Zb2 = parameters.loc[active_sim, 'Zb2']

            filedata[i] = line.split('(')[0] + '(' + str(Xb1) + ', ' + str(Yb1) + ', ' + str(Zb1) + ', ' + str(Xb2) + ', ' + str(Yb2) + ', ' + str(Zb2) + '); // bounds\n'

    # modify number of particles
    if 'N' in parameters.columns:
        filedata[0] = filedata[0].split('=')[0] + '= ' + str(int(parameters.loc[active_sim, 'N'])) + ',' + filedata[0].split(',')[-1]

    with open(attpcroot_dir + 'macro/Simulation/Charge_Dispersion/Mg20_test_sim_pag.C', 'w') as file:
        file.writelines(filedata)
    return None

In [90]:
def write_rundigi_sim_CD(parameters, active_sim, automation_dir=automation_dir, attpcroot_dir=attpcroot_dir):
    with open(automation_dir + 'rundigi_sim_CD.C', 'r') as file:
        filedata = file.readlines()

    # modify number of particles
    for i, line in enumerate(filedata):
        if ' N\n' == line.split('//')[-1]:
            if 'N' in parameters.columns:
                filedata[i] = line.split(',')[0] + ', ' + str(int(parameters.loc[active_sim, 'N'])) + '); // N\n'

    # modify Threshold
    for i, line in enumerate(filedata):
        if 'Threshold' in line.split('//')[-1]:
            if 'Threshold' in parameters.columns:
                threshold = parameters.loc[active_sim, 'Threshold']
            else:
                threshold = 87

            filedata[i] = line.split('(')[0] + '(' + str(threshold) + '); // Threshold\n'

    # modify Charge Dispersion Adjacent Pads
    for i, line in enumerate(filedata):
        if 'Charge Dispersion' in line.split('//')[-1]:
            if 'CD' in parameters.columns:
                filedata[i] = line.split('(')[0] + '(' + str(int(str(parameters.loc[active_sim, 'CD'])[0])) + '); // Charge Dispersion\n'

    # write file
    with open(attpcroot_dir + 'macro/Simulation/Charge_Dispersion/rundigi_sim_CD.C', 'w') as file:
        file.writelines(filedata)
    return None

In [91]:
def write_AtPulseGADGET(parameters, active_sim, automation_dir=automation_dir, attpcroot_dir=attpcroot_dir):
    # Charge Dispersion in AtPulseGADGET.h
    with open(automation_dir + 'AtPulseGADGET.h', 'r') as file:
        filedata = file.readlines()
    for i, line in enumerate(filedata):
        if 'CDH' in line.split('//')[-1]:
            # set value to CDH, or CD if CDH is not specified
            if 'CD' in parameters.columns:
                filedata[i] = line.split('=')[0] + '= ' + str(int(str(parameters.loc[active_sim, 'CD'])[0])) + '; // CDH\n'
                indicator_file('BUILD')
            
            if 'CDH' in parameters.columns:
                filedata[i] = line.split('=')[0] + '= ' + str(int(parameters.loc[active_sim, 'CDH'])) + '; // CDH\n'
                indicator_file('BUILD')

    with open(attpcroot_dir + 'AtDigitization/AtPulseGADGET.h', 'w') as file:
        file.writelines(filedata)
    return None

In [ ]:
def write_AtPulseGADGETcxx(parameters, active_sim, automation_dir=automation_dir, attpcroot_dir=attpcroot_dir):
    # Charge Dispersion approximation in AtPulseGADGET.cxx
    with open(automation_dir + 'AtPulseGADGET.cxx', 'r') as file:
        filedata = file.readlines()
    
    # if CD contains 'a', use approximation
    if 'CD' in parameters.columns:
        if 'a' in str(parameters.loc[active_sim, 'CD']):
            for i, line in enumerate(filedata):
                if 'TMath::Erf' in line:
                    filedata[i] = line.split('TMath::Erf')[0] + 'erfApproxWinitzki' + line.split('TMath::Erf')[1]
    
    with open(attpcroot_dir + 'AtDigitization/AtPulseGADGET.cxx', 'w') as file:
        file.writelines(filedata)
    return None


In [ ]:
def write_mediageo(parameters, active_sim, automation_dir=automation_dir, attpcroot_dir=attpcroot_dir):
    with open(automation_dir + 'media.geo', 'r') as file:
        filedata = file.readlines()
    changed = False
    
    # no current parameters to modify
    
    if changed: # create geo.temp to indicate geometry update
        os.system(f"touch {automation_dir}geo.temp")
    
    with open(attpcroot_dir + 'geometry/media.geo', 'w') as file:
        file.writelines(filedata)
    return None

In [ ]:
def write_geometry(parameters, active_sim, automation_dir=automation_dir, attpcroot_dir=attpcroot_dir):
    with open(automation_dir + 'GADGET_II.C', 'r') as file:
        filedata = file.readlines()
    
    changed = False
    
    if 'MediumGas' in parameters.columns:
        # replace all instances of 'GADGET_5IsoAr_800' with MediumGas
        for i, line in enumerate(filedata):
            if 'GADGET_P10_800' in line:
                if 'GADGET_P10_800' != parameters.loc[active_sim, 'MediumGas']:
                    filedata[i] = line.replace('GADGET_P10_800', parameters.loc[active_sim, 'MediumGas'])
                    changed = True
    
    if changed: # create geo.temp to indicate geometry update
        os.system(f"touch {automation_dir}geo.temp")
    
    with open(attpcroot_dir + 'geometry/GADGET_II.C', 'w') as file:
        file.writelines(filedata)
    return None

In [92]:
start_time = time.time()
while True: # WAIT LOOP
    stop_check(automation_dir)
    parameters = pd.read_csv(automation_dir + 'param.csv')
    
    if len(parameters[parameters['Status'] == 1]) > 0:
        indicator_file('STOP') # error if this file detects active simulations
    
    if not parameters['Sim'].is_unique:
        indicator_file('STOP') # sim names must be unique
    
    inactive_sims = parameters[parameters['Status'] == 0]
    if len(inactive_sims) == 0:
        indicator_file('WAIT') # no inactive simulations
    else:
        active_sim = inactive_sims.index[0] # first inactive simulation
        parameters.loc[active_sim, 'Status'] = 1
        if 'Time' in parameters.columns:
            parameters.loc[active_sim, 'Time'] = time.time()
        print(f"{parameters.loc[active_sim, 'Sim']}.h5")
        
        write_gadgetsimpar(parameters, active_sim)
        write_AtTPC20MgDecay_pag(parameters, active_sim)
        write_Mg20_test_sim(parameters, active_sim)
        write_rundigi_sim_CD(parameters, active_sim)
        write_AtPulseGADGET(parameters, active_sim)
        write_AtPulseGADGETcxx(parameters, active_sim)
        
        parameters.to_csv(automation_dir + 'param.csv', index=False)
        indicator_file('RUN')
        sys.exit(0)
    
    time.sleep(1) # wait 1 second before checking again
    
    if time.time() - start_time > 300:
        print('Timeout') # timeout after 5 minutes of waiting
        indicator_file('STOP')

ParticleTest.h5


SystemExit: 0

/user/jaros/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
